# ENGR 240: Parameters in Roots Problems - Michaelis-Menten Kinetics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%203/Worksheet%203-2_template.ipynb)

## Introduction

In many engineering applications, we encounter nonlinear equations that need to be solved for different parameter values. This worksheet focuses on how to handle changing parameters when using root-finding methods and explores the concept of "residuals" in numerical approximations.

### Learning Objectives
- Implement parameter-passing techniques in root-finding methods
- Compare Newton-Raphson method and `scipy.fsolve` for parametric roots problems
- Understand and calculate residuals in numerical approximations
- Visualize substrate concentration profiles using the Michaelis-Menten model
- Gain programming practice with functions, loops, and arrays

### Mathematical Background

The **Michaelis-Menten model** describes the kinetics of enzyme-mediated reactions. It is represented by the following equation:

$$S = S_0 - v_m t + k_s \ln\left(\frac{S_0}{S}\right)$$

where:
- $S$ = substrate concentration (mol/L)
- $S_0$ = initial substrate concentration (mol/L) at t = 0
- $v_m$ = maximum uptake rate (mol/L/d)
- $k_s$ = half saturation constant (mol/L)
- $t$ = time (days)

For this worksheet, we need to solve for $S$ given specific values of $S_0$, $v_m$, $k_s$, and $t$. Since $S$ appears on both sides of the equation, we can't solve for it directly. Instead, we need to rearrange the equation to find the roots:

$$f(S) = S - S_0 + v_m t - k_s \ln\left(\frac{S_0}{S}\right) = 0$$

## Setup and Imports

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import time  # for timing comparison

## Newton-Raphson Method with Parameters

The Newton-Raphson method is an iterative approach to finding roots. We've previously studied the basic implementation. The method uses the formula:

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

When working with parameters, we need to modify our implementation to accept additional parameter values.

### Implementation with Parameter Passing

Let's define our function and its derivative for the Michaelis-Menten model:

In [ ]:
def mm_function(S, S0, vm, ks, t):
    """
    Michaelis-Menten function whose root we seek.
    
    Parameters:
    -----------
    S : float
        Substrate concentration (mol/L)
    S0 : float
        Initial substrate concentration (mol/L)
    vm : float
        Maximum uptake rate (mol/L/d)
    ks : float
        Half saturation constant (mol/L)
    t : float
        Time (days)
        
    Returns:
    --------
    float : Function value at S
    """
    return S - S0 + vm * t - ks * np.log(S0 / S)

### Task 1: Derive and Implement the Derivative Function

Derive the derivative of the Michaelis-Menten function with respect to S:

$$\frac{d}{dS}\left[S - S_0 + v_m t - k_s \ln\left(\frac{S_0}{S}\right)\right]$$

Implement your derived formula in the `mm_derivative` function below. Test your implementation by running the Newton-Raphson method.

In [ ]:
# NOTE: You will need to derive and implement the derivative function.
# Remove the next line and add your implementation for mm_derivative

"""
STUDENT ACTIVITY:
1. Derive the derivative of mm_function with respect to S
2. Implement the derivative function below
"""

def mm_derivative(S, S0, vm, ks, t):
    """
    Derivative of the Michaelis-Menten function with respect to S.
    
    Parameters:
    -----------
    (Same as mm_function)
        
    Returns:
    --------
    float : Derivative value at S
    """
    # Replace this with your implementation
    pass

Now let's implement the Newton-Raphson method with parameter passing:

In [ ]:
def newton_raphson(func, dfunc, x0, tol=1e-8, max_iter=100, **params):
    """
    Newton-Raphson method to find a root of a function.
    
    Parameters:
    -----------
    func : function
        Function to find root of
    dfunc : function
        Derivative of the function
    x0 : float
        Initial guess
    tol : float
        Error tolerance
    max_iter : int
        Maximum number of iterations
    **params : dict
        Additional parameters to pass to the function
        
    Returns:
    --------
    tuple : (root, iterations, error, residual)
    """
    x = x0
    error = float('inf')
    iter_count = 0
    
    while error > tol and iter_count < max_iter:
        f_x = func(x, **params)
        df_x = dfunc(x, **params)
        
        if df_x == 0:
            raise ValueError("Derivative is zero. Cannot continue.")
        
        x_new = x - f_x / df_x
        
        # Calculate relative error
        error = abs((x_new - x) / x_new) if x_new != 0 else abs(x_new - x)
        
        x = x_new
        iter_count += 1
    
    # Calculate residual - the function value at our solution
    residual = abs(func(x, **params))
    
    return x, iter_count, error, residual

## Using scipy.fsolve with Parameters

SciPy's `fsolve` function is a powerful tool for finding roots of nonlinear equations. It implements a modified version of the Powell hybrid method, combining the benefits of Newton-Raphson and the secant method. This makes it robust for a wide range of problems.

### How fsolve Works

The `fsolve` function is designed to solve systems of nonlinear equations. Its algorithm works as follows:

1. It approximates the Jacobian matrix (which contains derivatives) using finite differences
2. Uses this approximation to determine a search direction
3. Takes steps in that direction, adapting the step size as needed
4. Continues iterating until convergence or reaching the maximum iterations

### Basic Syntax

The basic syntax of `fsolve` is:

```python
result = optimize.fsolve(func, x0, args=())
```

Where:
- `func` is the function to find the roots of
- `x0` is the initial guess
- `args` is a tuple of parameters to pass to the function

A key challenge is that `fsolve` requires the function to take the variable followed by additional parameters. When working with parameters, we need a way to pass these parameters while still maintaining the function signature required by `fsolve`.

There are several ways to handle this:

1. Using lambda functions
2. Using `functools.partial`
3. Creating a wrapper function
   
Let's see how to implement the first approach:

In [ ]:
def solve_with_fsolve(t, S0, vm, ks, x0):
    """
    Solve the Michaelis-Menten equation using scipy.fsolve.
    
    Parameters:
    -----------
    t : float
        Time (days)
    S0, vm, ks : float
        Model parameters
    x0 : float
        Initial guess
        
    Returns:
    --------
    tuple : (root, residual)
    """
    # Lambda function with parameters
    f = lambda S: mm_function(S, S0=S0, vm=vm, ks=ks, t=t)
    
    # Solve using fsolve
    root = optimize.fsolve(f, x0)[0]
    
    # Calculate residual
    residual = abs(f(root))
    
    return root, residual

## Problem Setup

Let's set up our problem with the following parameters:
- Initial substrate concentration (S0) = 10 mol/L
- Maximum uptake rate (vm) = 0.5 mol/L/d
- Half saturation constant (ks) = 2 mol/L
- Time range from 0 to 50 days in steps of 5 days

In [ ]:
# Set parameters
S0 = 10.0  # mol/L
vm = 0.5   # mol/L/d
ks = 2.0   # mol/L
t_values = np.arange(0, 51, 5)  # days

# Initial guess for S
initial_guess = 0.0001

# Define parameter dictionaries for both methods
newton_params = {'S0': S0, 'vm': vm, 'ks': ks}
fsolve_bracket = [0.0001, 10.01]  # Initial bracket for fsolve

## Calculating S vs t and Residuals

Now let's calculate S vs t and the associated residuals for both methods:

In [ ]:
# Arrays to store results
S_newton = np.zeros_like(t_values, dtype=float)
residuals_newton = np.zeros_like(t_values, dtype=float)
iterations_newton = np.zeros_like(t_values, dtype=int)

S_fsolve = np.zeros_like(t_values, dtype=float)
residuals_fsolve = np.zeros_like(t_values, dtype=float)

# Time execution
newton_time = 0
fsolve_time = 0

# Loop through each time value
for i, t in enumerate(t_values):
    # Newton-Raphson method
    newton_start = time.time()
    newton_params['t'] = t
    S_newton[i], iter_count, error, residuals_newton[i] = newton_raphson(
        mm_function, mm_derivative, initial_guess, **newton_params)
    iterations_newton[i] = iter_count
    newton_time += time.time() - newton_start
    
    # fsolve method
    fsolve_start = time.time()
    S_fsolve[i], residuals_fsolve[i] = solve_with_fsolve(t, S0, vm, ks, initial_guess)
    fsolve_time += time.time() - fsolve_start

# Print computation times
print(f"Newton-Raphson execution time: {newton_time:.6f} seconds")
print(f"fsolve execution time: {fsolve_time:.6f} seconds")

## Visualizing the Results

Let's create plots to visualize our results:

In [ ]:
# Create figure with subplots
fig, axes = plt.subplots(2, 1, figsize=(10, 12))

# Plot S vs t for both methods
axes[0].plot(t_values, S_newton, 'b-o', label='Newton-Raphson Method')
axes[0].plot(t_values, S_fsolve, 'r--x', label='fsolve Method')
axes[0].set_xlabel('Time (days)')
axes[0].set_ylabel('Substrate Concentration (mol/L)')
axes[0].set_title('Substrate Concentration vs. Time')
axes[0].grid(True)
axes[0].legend()

# Plot residuals
axes[1].semilogy(t_values, residuals_newton, 'b-o', label='Newton-Raphson Residuals')
axes[1].semilogy(t_values, residuals_fsolve, 'r--x', label='fsolve Residuals')
axes[1].set_xlabel('Time (days)')
axes[1].set_ylabel('Residual (log scale)')
axes[1].set_title('Residuals vs. Time')
axes[1].grid(True)
axes[1].legend()

plt.tight_layout()
plt.show()

# Print the results in a table
print("\nResults Table:")
print("=" * 75)
print(f"{'Time (days)':^12} | {'Newton-Raphson S':^15} | {'fsolve S':^15} | {'N-R Residual':^15} | {'fsolve Residual':^15}")
print("=" * 75)
for i, t in enumerate(t_values):
    print(f"{t:^12.1f} | {S_newton[i]:^15.6f} | {S_fsolve[i]:^15.6f} | {residuals_newton[i]:^15.2e} | {residuals_fsolve[i]:^15.2e}")

## Student Activities

Now it's your turn to work on the following tasks:

### Task 2: Implement a Different Method for Parameter Passing in fsolve

Implement one of the alternative methods for passing parameters to `fsolve` (either using `functools.partial` or creating a wrapper function).

In [ ]:
# Your implementation here

### Task 3: Modify for Multiple Initial Guesses

Write code to test how different initial guesses affect the convergence of both methods. Try several initial guesses and compare the results.

In [ ]:
# Your implementation here

### Task 4: Calculate Error Metrics

Calculate and plot the mean squared error between the solutions obtained by the Newton-Raphson method and `fsolve`. Is there a significant difference between the methods?

In [ ]:
# Your implementation here

### Task 5: Optimize Parameter Passing Strategy

For larger problems with many parameter values, efficient parameter passing becomes important. Explore and implement a strategy that would handle a large number of parameter values efficiently.

In [ ]:
# Your implementation here

## Conclusion

In this worksheet, we've explored how to handle parameters in root-finding problems using the Newton-Raphson method and `scipy.fsolve`. We've also examined the concept of residuals in numerical approximations and compared the efficiency and accuracy of different approaches.

Key takeaways:
1. Parameter passing in numerical methods can be handled through function arguments or keyword arguments
2. Lambda functions provide a clean way to parameterize functions for `scipy.fsolve`
3. Residuals help us understand the accuracy of our numerical solutions
4. For complex problems, specialized libraries like SciPy offer robust and efficient solutions

As you continue with your engineering studies, you'll find that many real-world problems require solving parameterized equations repeatedly for different conditions - the techniques covered here provide a foundation for addressing these challenges.